In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2
scikit-learn-intelex==2021.20210714.120553


In [13]:
import pickle
import pandas as pd
from pathlib import Path

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [14]:
year = 2021
month = 2
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year}-{month:02d}.parquet')

In [15]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

Q1

In [16]:
print(y_pred.mean())

16.191691679979066


Q2

In [17]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [29]:
df_result = pd.DataFrame({'ride_id': df['ride_id'], 'predicted_duration': y_pred})
output = Path().resolve() / f'fhv_tripdata_{year:04d}_{month:02d}_predictions.parquet'
df_result.to_parquet(
    output,
    engine='pyarrow',
    compression=None,
    index=False
)

In [30]:
print(f"File size: {output.stat().st_size / 1024 ** 2:.2f} MB")

File size: 18.80 MB
